In [1]:
import pandas as pd
import re
from rapidfuzz import process, fuzz

In [2]:
# Load the input CSV file
input_file = 'geologiclog_freeform.csv'  # Update with your file path

encodings = ['utf-8', 'latin1', 'cp1252', 'utf-16']  # Add more encodings if needed

# Attempt to read the file with different encodings
for encoding in encodings:
    try:
        df = pd.read_csv(input_file, encoding=encoding)
        break
    except UnicodeDecodeError:
        continue
else:
    raise Exception("Failed to read the lookup table. Please check the file encoding.")

# Check if the 'DESCRIPTION' column exists
if 'DESCRIPTION' not in df.columns:
    raise KeyError("The 'DESCRIPTION' column does not exist in the DataFrame.")



C:\Users\betebari\AppData\Local\Temp\ipykernel_24104\2993274231.py:9: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(input_file, encoding=encoding)


In [3]:
# Function to extract text within parentheses
def extract_uscs(text):
    if isinstance(text, str):
        match = re.search(r'\((.*?)\)', text)
        if match:
            return match.group(1)
    return text

# Apply the function to the 'DESCRIPTION' column and create a new 'USCS2' column
df['USCS2'] = df['DESCRIPTION'].apply(extract_uscs)

In [5]:
# List of USCS codes to look for
uscs_codes = [
    '(ML)', '(GP)', '(SP-SM)', '(GC)', '(CL)', '(CH)', '(MH)', '(GP-GM)',
    '(GP-GC)', 'GM', '(ml)', 'SP-SM', '(SC-SM)', 'CH', '(SC)',
    'SM-SC','SM-ML','SM-GP','SP-GP', 'SW-GW','SC-GC','SC-ML','SP/GP/CL','CL/SP',
    'GW-GM','OH/CH','CL/CH', 'Cl','SW-SM','SW-SC','SP-SM','SP-SC','OL','GW-GC','GP-SP',
    'GP-GM','GP-GC','GP-CG','CL-ML','CH+GP', 
]

In [4]:
# Convert 'DESRIPTION' column to lowercase and clean it
df['Category'] = df['DESCRIPTION'].astype(str).str.lower().str.strip().apply(lambda x: x.replace('\n', ' ').replace('\r', '')[:250])

# Function to replace specified characters with spaces
def convert_chars(text):
    for char in ['/', '&', '-', '|', ':', '+', '?', '>']:
        text = text.replace(char, ' ')
    return text

df['Category'] = df['Category'].apply(convert_chars)

In [7]:
# Define a dictionary for category conversions
conversions = {
    # Add all conversion rules here
    'dg': 'decomposed granite',
    'd,g.': 'decomposed granite',
    'D/G': 'decomposed granite',
    'd.g': 'decomposed granite',
    'd & g': 'decomposed granite',
    'D.G.': 'decomposed granite',
    'dgf': 'decomposed granite',
    'quartz': 'decomposed granite',
    'cemented gravel': 'conglomerate',
    'sand stone': 'sandstone',
    'overburden': 'soil',
    'small': 'fine',
    'sand clay': 'clayey sand',
    'sand and clay': 'clayey sand',
    'sand and gravel': 'gravely sand',
    'gravel clay': 'clayey gravel',
    'clay gravel': 'gravely clay',
    'clay sand': 'sandy clay',
    'clay sandy': 'sandy clay',
    'sand gravel': 'gravely sand',
    'gravel and sand': 'sandy gravel',
    'sand & gravel': 'gravely sand',
    'mixed sands': 'poorly graded sand',
    'firm': 'compacted',
    'hard': 'compacted',
    'heavy': 'compacted',
    'very stiff': 'compacted',
    'blacksand': 'sand',
    'large': 'coarse',
    'big': 'coarse',
    'solid': 'compacted',
    'sand-coarse': 'coarse sand',
    'gravel with clay': 'clayey gravel',
    'soft': 'fine',
    'course': 'coarse',
    'clay loam': 'clayey sand',
    'chunky': 'coarse',
    'dirt': 'soil',
    'Sand stone': 'sandstone',
    'clay-blue': 'clay',
    'blue clay': 'clay',
    'channel': 'coarse',
    'grit': 'gravel',
    'cobble': 'gravel',
    'ashÂ': 'ash',
    'SERP.': 'serpentine',
    'SERP': 'serpentine',
    'Adobe': 'clay',
    'gray ash': 'ash',
    'mud': 'clay',
    'Sand-Clay': 'clayey sand',
    'sand,clay': 'clayey sand',
    'Sand M': 'sand medium',
    'Sand M-C': 'sand medium coarse',
    'Hard Pen': 'hardpan',
    'Birds Eye': 'gravel',
    'Birds eye': 'gravel',
    'Crystalized Minerals': 'gravel',
    'sand&clay': 'clayey sand',
    'Hard Clay': 'claystone',
    'IONE': 'sandstone',
    'Marsh Deposits': 'organic soil',
    'loose': 'fine',
    'GrayWackey': 'sandstone',
    'Gray Wackey': 'sandstone',
    'Greywacke': 'sandstone',
    'SILTYCLAY': 'silty clay',
    'mudflow': 'clay',
    'MERTON': 'Mehrton frm',
    'Ground': 'soil',
    'Overberdon': 'soil',
    'Crystallized Minerals': 'granite',
    'PEAT (PT)': 'peat',
    'stopped drilling': 'claystone',
    'TOP SOIL': 'soil',
    'grey': 'gray',
    'boulders': 'boulder',
    'CINDERS': 'cinder',
    'COBBLES': 'cobble',
    'tuft': 'tuff',
    'joint': 'fractured',
    'bluish': 'blue',
    'quarts' : 'quartz',
    'Over burden' : 'soil',
    'colluvium':'gravel',
    'cobbles' : 'gravel',
    'boulders': 'gravel',
    'alluvium' : 'sandy gravel',
    'SHIST' : 'schist',
    'Sandstone': 'Sand-stone',
    'basalt': 'basaltic',
    'Schist': 'Schistoze',
    # Add more conversions as needed
}

# Function to apply conversions
def apply_conversions(text, conversions):
    for key, value in conversions.items():
        text = re.sub(r'\b{}\b'.format(re.escape(key)), value, text)
    return text


In [8]:

# Apply conversions to the 'Category' column
df['Category'] = df['Category'].apply(lambda x: apply_conversions(x, conversions))

# Define a list of correct words
correct_words = [
    'red', 'green', 'black', 'brown', 'yellow', 'sand', 'gravel', 'clay', 'silt', 'greenstone',
    'sandy', 'gravely', 'clayey', 'silty', 'granite', 'basalt', 'volcanics', 'volcanic',
    'soil', 'organic', 'peat', 'sand-stone', 'conglomerate', 'siltstone', 'diorite', 'laminated',
    'decomposed', 'fractured', 'rock', 'shale', 'claystone', 'fine', 'medium', 'pumice',
    'coarse', 'grained', 'sticky', 'cobble', 'latite', 'graded', 'loam', 'rock', 'poorly graded',
    'cemented', 'shell', 'quartz', 'quartzite', 'metasediment', 'slate', 'schist', 'solid',
    'granodiorite', 'hardpan', 'chert', 'igneous', 'Serpentine', 'phyllite', 'limestone',
    'mudstone', 'andesite', 'basaltic', 'Schistoze',
    
]

# Function to fix typos using fuzzy matching
def fix_typos(text, choices):
    words = text.split()
    fixed_words = []
    for word in words:
        cleaned_word = re.sub(r'[^a-zA-Z0-9,%]', '', word)  # Remove non-alphabetic characters except numbers, commas, and percentage signs
        if cleaned_word.strip():  # Check if the cleaned word is not empty
            best_match = process.extractOne(cleaned_word, choices, scorer=fuzz.token_sort_ratio)
            fixed_words.append(best_match[0])
        else:
            fixed_words.append(word)
    return ' '.join(fixed_words)

# Apply the function to the 'Category' column
df['Category'] = df['Category'].apply(lambda x: fix_typos(x, correct_words))

# Define the words to remove
words_to_remove = [
    'red', 'green', 'black', 'brown', 'gray', 'grayish', 'reddish', 'yellow', 'dark', 'iron', 'H2O',
    'light', 'tan', 'colored', 'blue', 'mix', '@', '&', '*', 'grey', 'dry', 'B&W', 'blind',
    'see', 'attached.', 'S&P', 'semi', 'please', 'log', 'geologic', 'log.', 'for', 'details.', 
    'find', 'below.', 'logging', 'by', 'core', 'rubble', 'bored', 'Drilling', 'LLC.', 'old',
    'sample', 'too', 'small.', 'only', 'show', 'comp', 'sheet', 'attached', 'numbers', 'XX', 
    'pieces', 'rock.', 'clay/gravel.', 'as', 'above.', 'PT.', 'test', 'permit.', 'per','olive',  
    'documents.', 'pile', 'cap', 'elev.', '@-', '&-', '*-', 'gravel.sandy.', 'compacted', 
    'silty.sandy', 'same', 'from', 'start', 'sample.', 'same.', 'all', 'piece', 'fill', 
    'off', 'off-', 'note', 'N/A', 'loose', 'hard', 'pothole', 'open', 'void','control', 
]

# Function to remove specified words
def remove_words(text, words):
    pattern = re.compile(r'\b(?:{})\b'.format('|'.join(map(re.escape, words))), flags=re.IGNORECASE)
    return pattern.sub('', text).strip()

# Apply the function to the 'Category' column
df['Category'] = df['Category'].apply(lambda x: remove_words(x, words_to_remove))


# Function to extract USCS code from description and update USCS column
def extract_uscs(description):
    uscs = None
    for code in uscs_codes:
        # Check if the code is in the description
        if code.lower() in description.lower():
            uscs = code.upper()
            # Remove the code from the description
            description = re.sub(re.escape(code), '', description, flags=re.IGNORECASE)
            break
    return description.strip(), uscs

# Apply the function to the DataFrame
df['Category'], df['USCS'] = zip(*df['Category'].apply(extract_uscs))

In [9]:
# Define the words to  keep
words_to_keep = [
    'sand', 'gravel', 'clay', 'silt', 'sandy', 'gravely', 'clayey', 'silty', 'mudstone',
    'granite', 'basalt', 'volcanics', 'volcanic', 'hard', 'compacted', 'boulder', 'fine', 'coarse',
    'lava', 'medium', 'cemented', 'weathered', 'decomposed', 'schist', 'ash', 'latite', 'SC',
    'fracture', 'fractured', 'chert', 'soil', 'shale', 'breccia', 'ML', 'GM', 'GC', 'SM',
    'CH', 'Cl/ML', 'OH', 'SP', 'SP-SM', 'GP-GM', 'limestone', 'graded', 'lava', 'loam', 'rock',
    'poorly graded', 'greenstone', 'sandstone', 'siltstone', 'claystone', 'cobblestone', 'mudstone',
    'Serpentine', 'organics', 'cemented', 'diorite', 'cinder', 'pumice', 'Cemented Gravel',
    'conglomerate', 'Gray Wacke','Quartz', 'GP','cinders', 'SC','Peat' ,'quartzite','metasediment',
    'granodiorite', 'hardpan', 'chert', 'slate','GPM' , 'andesite',
]

In [10]:
# Define a list of words that should not be replaced
words_to_exclude = [
    'blue','Gray',  'grey', 'SEE ATTACHED',  'Salt Pepper','Black / White', 'GPM', 'BROWN & GRAY',
    'Asphalt Base', 'light', 'ash', 'See attached.', 'Asphalt', 'white',  'gray', 
]
# Apply replacement only to characters, not words
for word in words_to_exclude:
    df['Category'] = df['Category'].str.replace(r'\b{}\b'.format(word), word)

In [11]:
# Define a function to revert incorrect replacements, if necessary
def revert_incorrect_replacements(text):
    # Map of incorrect replacements to their original words
    incorrect_replacements = {
        'latite': 'blue',
        'basalt': 'Asphalt',
        'pumice' : 'GPM',
        'gravely': 'gray',
        'gravely': 'grey',
        'hard': 'decomposed',
        'diorite' : 'joint',
        'sandstone' : 'top',
        
        # Add other incorrect replacements if needed
}
    for incorrect, original in incorrect_replacements.items():
        text = re.sub(r'\b{}\b'.format(incorrect), original, text, flags=re.IGNORECASE)
    return text

# Apply the revert function
df['Category'] = df['Category'].apply(revert_incorrect_replacements)

In [12]:
# Escape special characters in words_to_remove
escaped_words_to_remove = [re.escape(word) for word in words_to_remove]

# Create a regular expression pattern for words to remove
pattern = r'\b(?:{})\b'.format('|'.join(escaped_words_to_remove))

# Function to remove unwanted words
def remove_words(text):
    return re.sub(pattern, '', text)

# Apply the function to the 'Category' column
df['Category'] = df['Category'].apply(remove_words)

# Function to keep only relevant words
def keep_relevant_words(text, words_to_keep):
    words = text.split()
    kept_words = [word for word in words if word in words_to_keep or re.match(r'^[0-9,%]+$', word)]
    return ' '.join(kept_words)

df['Category'] = df['Category'].apply(lambda x: keep_relevant_words(x, words_to_keep))

# Fill blank rows with a placeholder value 'unknown'
df['Category'] = df['Category'].apply(lambda x: 'unknown' if not x.strip() else x)

In [13]:
# Define a dictionary for USCS group symbols
uscs_conversions = {
    'poorly graded gravel': 'GP',
    'GP': 'GP',
    'GW': 'GW',
    'well graded gravel': 'GW',
    'silty gravel': 'GM',
    'well graded sand': 'SW',
    'SW': 'SW',
    'sandy clay' : 'CL',
    'gravely sand': 'SP',
    'Sediment' : 'SP',
    'alluvium': 'GP',
    
    'poorly graded sand': 'SP',
    'SP': 'SP',
    'silty sand': 'SM',
    'SM': 'SM',
    'sandy loam': 'SM',
    'clayey gravel': 'GC',
    'GC': 'GC',
    'cobble clay': 'GC',
    'clayey sand': 'SC',
    'clay sand': 'CL',
    'SC': 'SC',
    'clayey silt': 'ML',
    'loam' : 'ML' ,
    'ML' : 'ML' ,
    'peat' : 'PT',
    'organics': 'PT',
    'sticky clay': 'CH',
    'CH': 'CH',
    'fat clay': 'CH',
    'fat clay' : 'CH',
    'adobe' : 'Cl',
    
    'lean clay' : 'CL',
    'silty clay' : 'CL',
    'lean clay': 'CL',
    'sandy clay' : 'CL',
    'hardpan' : 'ML',
    'silt' : 'ML',
    'sandy silt' : 'ML',
    'organic soil': 'OH',
    
    'OH' : 'OH',
    'silty soil': 'OH',
    'cobble' : 'GW',
    'boulder': 'GW',
    'decomposed granite': 'GW',
    'SP-SM' : 'SP-SM',
    'GP-GM': 'GP-GM',
    'Cl/ML' : 'CL-ML',
    'sand': 'SW',
    'gravel': 'GW',
    'clay' : 'CH',
    'soil' : 'OH',
    'top soil' : 'OH',
    'topsoil' : 'OH',
    'SC' : 'SC',
    'sandy' : 'SW',
    
    'basalt' : 'volcanic',
    'basaltic' : 'volcanic',
    'pumice' : 'volcanic',
    'latite' : 'volcanic',
    'volcanics' : 'volcanic',
    'volcanic' : 'volcanic',
    'cinder' : 'volcanic',
    'ash' : 'volcanic',
    'tufa': 'volcanic',
    'tuff': 'volcanic',
    
    'granite' : 'igneous rock',
    'diorite': 'igneous rock',
    'Quartz': 'igneous rock',
    'Gabro': 'igneous rock',
    'gabbro': 'igneous rock',
    'quartzite': 'igneous rock',
    'granodiorite': 'igneous rock',
    'igneous': 'igneous rock',
    'andesite': 'igneous rock',
    
    'greenstone' : 'metamorphic rock',
    'slate' : 'metamorphic rock',
    'schist' : 'metamorphic rock',
    'Serpentine':'metamorphic rock',
    'metasediment':'metamorphic rock',
    'phylite':'metamorphic rock',
    'phyllite':'metamorphic rock',
    'argillite':'metamorphic rock',
    
    'sandstone' : 'sedimentary rock',
    'conglomerate' : 'sedimentary rock',
    'shale' : 'sedimentary rock',
    'siltstone' : 'sedimentary rock',
    'rock' : 'sedimentary rock',
    'Gray Wacke': 'sedimentary rock',
    'limestone' : 'sedimentary rock',
    'siltstone': 'sedimentary rock',
    'claystone': 'sedimentary rock',
    'cobblestone': 'sedimentary rock',
    'mudstone': 'sedimentary rock',
    'Cemented gravel':'sedimentary rock',
    'chert' : 'sedimentary rock',
    
    'fractured' :'double-porosity rock',
    'fracture' :'double-porosity rock',
    'fractured rock':'double-porosity rock',
    'rock fractured':'double-porosity rock',

    # Add more USCS conversions as needed
}

In [14]:
# Function to assign USCS group symbols based on Category
def assign_uscs(text, uscs_conversions):
    for key, value in uscs_conversions.items():
        if re.search(r'\b{}\b'.format(re.escape(key)), text):
            return value
    return 'unknown'

# Add a new column 'USCS' and apply the conversions
df['USCS2'] = df['Category'].apply(lambda x: assign_uscs(x, uscs_conversions))

In [15]:
# Save the updated DataFrame to a new CSV file
output_file = 'updated_geologiclog_freeform.csv'
df.to_csv(output_file, index=False)

print(f"Updated CSV file saved as '{output_file}'")

Updated CSV file saved as 'updated_geologiclog_freeform.csv'
